# Scikit-learn Iris Classifier - Local Example

_**Train and export a scikit-learn classifier for the [Iris data set](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) dataset: Performing all storage and computation locally on the notebook.**_

This notebook works well with the `Python 3 (Data science)` kernel on SageMaker Studio, or `conda_python 3` on classic SageMaker Notebook Instances.

---

The [Iris dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/) is hosted in the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) and maintain 622 data sets.

>❓*Can you figure out how to re-create this notebook's workflow using SageMaker more effectively?*

## Contents

1. **[Prepare the Data](#Prepare-the-Data)**
1. **[Data processing and training](#Data-processing-and-training)**
1. **[Build and fit the Model](#Build-and-fit-the-Model)**
1. **[Save the Trained Model](#Save-the-Trained-Model)**
1. **[Explore Results](#Explore-Results)**

See the accompanying **Instructions** notebook for more guidance!

In [2]:
import argparse
import numpy as np
import os
import pandas as pd
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import joblib

/opt/conda/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## Prepare the Data

Now let's download the Iris data to your local directory.

In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2022-06-23 05:26:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551 (4.4K) [application/x-httpd-php]
Saving to: ‘iris.data’

iris.data           100%[===================>]   4.44K  --.-KB/s    in 0s      

2022-06-23 05:26:31 (129 MB/s) - ‘iris.data’ saved [4551/4551]



In [4]:
data = data = pd.read_csv('iris.data', 
                   names=['sepal length', 'sepal width', 
                          'petal length', 'petal width', 
                          'label'])
data.head()

,sepal length,sepal width,petal length,petal width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
#split the data into train and test
train,test= np.split(data.sample(frac=1, random_state=22), [int(0.7 * len(data))])
train.head()

#write your csv files to the local
train.to_csv("train.csv")
test.to_csv("test.csv")

## Data processing and training
- we need to convert the string labels to numeric labels to fit in our SkLearn model
- we need to sperate out the features from target variable and define train, test and their labels
- we also would like to standardise the features before fitting them into the model

In [6]:
# Dictionary to encode labels to codes
label_encode = {
    'Iris-virginica': 0,
    'Iris-versicolor': 1,
    'Iris-setosa': 2
}

# Dictionary to convert codes to labels
label_decode = {
    0: 'Iris-virginica',
    1: 'Iris-versicolor',
    2: 'Iris-setosa'
}

# sperate out the features from target variable and define train, test and their labels
train = pd.read_csv('train.csv',index_col=0, engine="python")
y_train= train['label'].map(label_encode)
X_train =  train.drop(["label"], axis=1)
    
test = pd.read_csv('test.csv',index_col=0, engine="python")
y_test= test['label'].map(label_encode)
X_test =  test.drop(["label"], axis=1)


## Build and fit the Model

The model chosen from the Scikit- learn classifiers, is the widely used a random forest model and takes the features and labels as input and returns the predicted lable or the probabilities (if chosen) as output.
Scikit-learn makes fitting and evaluating the model straightforward enough


In [7]:
#train the logistic regression model
n_estimators= 100
min_samples_leaf= 3
model = RandomForestClassifier(
        n_estimators=n_estimators,
        min_samples_leaf=min_samples_leaf)
model.fit(X_train, y_train)
model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Save the Trained Model

We use Joblib to save the model and then load it for prediction.


In [8]:
#use Joblib to save the model 
# see scikit learn documentation here:https://scikit-learn.org/stable/model_persistence.html
joblib.dump(model, "model.joblib")

['model.joblib']

Let's Explore Results

In [9]:
# load the model using joblib
loaded_model = joblib.load("model.joblib")

#get the data to predict
result = loaded_model.predict(X_test)
results=' | '.join([label_decode[t] for t in result])
results

'Iris-versicolor | Iris-versicolor | Iris-virginica | Iris-versicolor | Iris-virginica | Iris-setosa | Iris-setosa | Iris-virginica | Iris-versicolor | Iris-versicolor | Iris-setosa | Iris-versicolor | Iris-setosa | Iris-virginica | Iris-virginica | Iris-versicolor | Iris-virginica | Iris-setosa | Iris-virginica | Iris-virginica | Iris-setosa | Iris-versicolor | Iris-versicolor | Iris-setosa | Iris-virginica | Iris-setosa | Iris-versicolor | Iris-setosa | Iris-virginica | Iris-setosa | Iris-virginica | Iris-virginica | Iris-setosa | Iris-virginica | Iris-setosa | Iris-versicolor | Iris-setosa | Iris-setosa | Iris-virginica | Iris-versicolor | Iris-virginica | Iris-virginica | Iris-setosa | Iris-virginica | Iris-virginica'

All done!
